### Import Libraries

In [1]:
from netgen.geom2d import *
import scipy
import scipy.sparse.linalg
from ngsolve import *
from ngsolve.webgui import Draw

### Set Parameters

In [13]:
a = 2              # Length of rectangle in x direction 
b = 1             # Length of rectangle in y direction 

maxh = 0.1   # Mesh-size
order = 1      # Order of Polynomials
nr_eigs = 100     # Number of Eigenvalues to approximate

### Create the Rectangle

In [14]:
geo = SplineGeometry()
geo.AddRectangle((0,0), (a,b), bcs=["b","r","t","l"])
mesh = Mesh(geo.GenerateMesh(maxh=maxh))
Draw(mesh)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2009-14-g847138235', 'mesh_dim': 2, 'order2d': 1, 'order3d': 1,…

### finite element space and bilinear form

In [15]:
fes = H1(mesh, complex=True, order=order)

eigenvec = GridFunction(fes,multidim=nr_eigs)

u = fes.TrialFunction()
v = fes.TestFunction()

a = BilinearForm (fes)
a += SymbolicBFI (grad(u)*grad(v))

b = BilinearForm (fes)
b += SymbolicBFI (u*v)

a.Assemble()
b.Assemble()

A=a.mat
B=b.mat

print(fes.ndof)


263


### eigenvalue solver via scipy

In [16]:
shift = 4

shifted = a.mat.CreateMatrix()
shifted.AsVector().data = a.mat.AsVector() - (shift**2)*b.mat.AsVector()
invshifted = shifted.Inverse(freedofs=fes.FreeDofs())

tmp1 = eigenvec.vec.CreateVector()
tmp2 = eigenvec.vec.CreateVector()
def matvec(v):
    tmp1.FV().NumPy()[:] = v
    tmp2.data = b.mat * tmp1
    tmp1.data = invshifted * tmp2
    return tmp1.FV().NumPy()

A = scipy.sparse.linalg.LinearOperator( (a.mat.height,a.mat.width), matvec)
mu, vecs = scipy.sparse.linalg.eigs(A, nr_eigs)

lamsquared=shift**2+1/mu
lams=[sqrt(l) for l in lamsquared]
print("resonances:")
for i in range (0,len(lams)):
    print(i,": ",lams[i], " squared: ", lamsquared[i])  


resonances:
0 :  3.5270631629044065  squared:  (12.440174555117235-1.7105549519753929e-16j)
1 :  4.471955595411878  squared:  (19.99838684733561+1.341327966776462e-16j)
2 :  4.744005583667814  squared:  (22.505588977871398+2.8717163229860597e-16j)
3 :  3.151850764112277  squared:  (9.934163239235144-9.842500236454905e-16j)
4 :  3.1510012444197635  squared:  (9.928808842334899-6.034923631222385e-16j)
5 :  1.5719521225192916  squared:  (2.4710334754929058+1.1733942047924859e-16j)
6 :  2.1490760299348766e-07  squared:  (4.618527782440651e-14+2.2612635896946143e-15j)
7 :  5.7221572320678895  squared:  (32.74308338850685+2.020378487503845e-15j)
8 :  6.366273005370897  squared:  (40.5294319789142-2.315866972896634e-15j)
9 :  6.358132518131315  squared:  (40.42584911811886+8.085737281835806e-17j)
10 :  6.566628387789881  squared:  (43.120608383327934-7.938031354008079e-16j)
11 :  7.135622186157851  squared:  (50.91710398358814+1.7025580037207766e-15j)
12 :  7.141111740757764  squared:  (50.99

C:\Users\Lothar Nannen\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:19: ComplexWarning: Casting complex values to real discards the imaginary part


### copy and plot eigenfunctions

In [17]:
for i in range(nr_eigs):
    eigenvec.vecs[i].FV().NumPy()[:] = vecs[:,i]

Draw(eigenvec)

NGSWebGuiWidget(value={'ngsolve_version': '6.2.2009-14-g847138235', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2,…